In [17]:
from KGS import * # Utilizing my own package for data analysis. Remember to also send enviroment when sharing. 
import scipp as sc
from IPython.display import Image
from TubeMeasurment_mccode import BIFROST_measurement
mpl.rcParams['font.size']        = 16
import ipywidgets as ipw


In [18]:
BIFROST_measurement.load_scan('BIFROST_PSC_0p3ms_magnon_v3_1T','Magnon_v3_0p3ms_1T_reduced',folder_path='./Magnon_0p3ms_1T/', set_Elim=(-0.5,9))

Energier =  [74.62994    73.63964599 72.66770834 ... -3.49529873 -3.49791895
 -3.50053238]
shape matrix = (261078, 5)
Energier =  [74.62994    73.63964599 72.66770834 ... -3.49529873 -3.49791895
 -3.50053238]
shape matrix = (261078, 5)
Energier =  [74.62994    73.63964599 72.66770834 ... -3.49529873 -3.49791895
 -3.50053238]
shape matrix = (261078, 5)
Energier =  [74.62994    73.63964599 72.66770834 ... -3.49529873 -3.49791895
 -3.50053238]
shape matrix = (261078, 5)
Energier =  [74.62994    73.63964599 72.66770834 ... -3.49529873 -3.49791895
 -3.50053238]
shape matrix = (261078, 5)
Energier =  [74.62994    73.63964599 72.66770834 ... -3.49529873 -3.49791895
 -3.50053238]
shape matrix = (261078, 5)
Energier =  [74.62994    73.63964599 72.66770834 ... -3.49529873 -3.49791895
 -3.50053238]
shape matrix = (261078, 5)
Energier =  [74.62994    73.63964599 72.66770834 ... -3.49529873 -3.49791895
 -3.50053238]
shape matrix = (261078, 5)
Energier =  [74.62994    73.63964599 72.66770834 ... -3.

## Organize in scipp array

In [19]:
settings = np.loadtxt('Magnon_v3_0p3ms_1T_reduced').T

#settings = np.loadtxt('Magnon_v3_1ms_reduced').T

#settings = np.loadtxt('Magnon_v3_1ms_reduced').T

In [20]:
qx = sc.array(dims=["row"], values=settings[3], unit="Å^(-1)")
qy = sc.array(dims=["row"], values=settings[4], unit="Å^(-1)")
deltaE = sc.array(dims=["row"], values=settings[2], unit="meV")

recording = sc.DataArray(
    data=sc.array(dims=["row"], values=settings[0], unit="arb. units", variances=settings[1]**2),  #, variances=settings[1]
    coords={"qx": qx, "qy": qy, "deltaE":deltaE}
)
recording

<scipp.DataArray>
Dimensions: Sizes[row:52737756, ]
Coordinates:
* deltaE                    float64            [meV]  (row)  [8.96664, 8.89343, ..., -0.481309, -0.494836]
* qx                        float64           [1/Å]  (row)  [0.566949, 0.56151, ..., -0.897541, -0.898333]
* qy                        float64           [1/Å]  (row)  [1.18015, 1.17605, ..., 1.99716, 1.9951]
Data:
                            float64      [arb. unit]  (row)  [0, 0, ..., 0, 0]  [0, 0, ..., 0, 0]

In [21]:
sc.show(recording)

In [7]:
qx_bins = round((max(recording.coords['qx'].values) - min(recording.coords['qx'].values))/0.04)
qy_bins = round((max(recording.coords['qy'].values) - min(recording.coords['qy'].values))/0.04)
E_bins = round((max(recording.coords['deltaE'].values) - min(recording.coords['deltaE'].values))/0.1)

print(qx_bins, qy_bins, E_bins)
print(max(recording.coords['qx'].values))

112 87 95
3.304249


In [22]:
da = recording.bin(qx=qx_bins, qy=qy_bins, deltaE=E_bins)
sliced = da.bins.mean()
#sliced.values[(da.bins.size() == sc.index(0)).values] = np.nan
sliced.save_hdf5(filename='Magnon_v3_0p3ms_1T.hdf5')

sc.show(da)

/Users/kristinekrighaar/opt/anaconda3/lib/python3.9/site-packages/scipp/core/deprecation.py:7: VisibleDeprecationWarning: sc.DataArray.attrs has been deprecated and will be removed in Scipp v24.12.0. The deprecation includes sc.DataArray.meta and sc.DataArray.drop_attrs. For unaligned coords, use sc.DataArray.coords and unset the alignment flag. For other attributes, use a higher-level data structure.
  warnings.warn(


## qx,qy plane cuts

In [ ]:

qx_min = 0 * sc.Unit('Å^(-1)')
qx_max = 2 * sc.Unit('Å^(-1)')

qy_min = -2 * sc.Unit('Å^(-1)')
qy_max = 4.0 * sc.Unit('Å^(-1)')


def update_plot(amp):
    E_layer = amp * sc.Unit('meV')
    plt.figure(figsize=(6,4))
    p1 = sliced['deltaE', E_layer]['qy', qy_min:qy_max].transpose()#.plot(cmap='jet')
    im = plt.pcolormesh(p1.coords['qx'].values, p1.coords['qy'].values, p1.values, cmap='jet',  vmin=0, vmax=2) # vmin=0, vmax=3e-5
    cbar = plt.colorbar(im)
    cbar.set_label('I [a.u.]')
    plt.xlim(-2,3)
    plt.ylim(0,4)
    plt.xlabel('$q_x$ [1/Å]')
    plt.ylabel('$q_y$ [1/Å]')
    plt.title(f"$\Delta E =$ {amp:.2f} meV")
    #plt.savefig('Pb_phonon_BIFROST_qxqy_mid.pdf', format='pdf', bbox_inches='tight')
    

w = ipw.FloatSlider( 
    value=6.19,
    min=min(sliced.coords['deltaE'].values),
    max=9.0,
    step=(sliced.coords['deltaE'].values[1]-sliced.coords['deltaE'].values[0]),
    description="Delta_E:",
    disabled=False,
    continuous_update=True,
    orientation="horizontal",
    readout=True,
    readout_format=".2f",
)
# Display the slider

ipw.interactive(update_plot, amp=w)

interactive(children=(FloatSlider(value=6.19, description='Delta_E:', max=9.0, min=-0.499966, step=0.099906431…

## Cut in the qx plane

In [ ]:


qxE = recording.bin(qx=qx_bins, qy=qy_bins, deltaE=E_bins)
sliced_qxE = qxE.bins.mean()

qx_min = -2 * sc.Unit('Å^(-1)')
qx_max = 2 * sc.Unit('Å^(-1)')

E_min = -0.05 * sc.Unit('meV')
E_max = 9.0 * sc.Unit('meV')


def update_plot_qxE(amp):
    qy_layer = amp * sc.Unit('Å^(-1)')
    plt.figure(figsize=(6,4))
    p1 = sliced_qxE['qy', qy_layer]['deltaE', E_min:E_max].transpose()#.plot(cmap='jet')
    im = plt.pcolormesh(p1.coords['qx'].values, p1.coords['deltaE'].values, p1.values, cmap='jet', vmin=0, vmax=3) # norm=colors.LogNorm()
    cbar = plt.colorbar(im)
    cbar.set_label('I [a.u.]')
    plt.xlim(-2,2.5)
    plt.ylim(0,9)
    plt.xlabel('$q_x$ [1/Å]')
    plt.ylabel('$\Delta E$ [meV]')
    plt.title(f"$q_y =$ {amp:.2f} Å$^{-1}$")
    #plt.savefig('Pb_phonon_BIFROST_qxE_high.pdf', format='pdf', bbox_inches='tight')
    

w_qxE = ipw.FloatSlider( 
    value=2.73,
    min=min(sliced_qxE.coords['qy'].values),
    max=4,
    step=(sliced_qxE.coords['qy'].values[1]-sliced_qxE.coords['qy'].values[0]),
    description="q_y:",
    disabled=False,
    continuous_update=True,
    orientation="horizontal",
    readout=True,
    readout_format=".2f",
)
# Display the slider



ipw.interactive(update_plot_qxE, amp=w_qxE)

interactive(children=(FloatSlider(value=2.73, description='q_y:', max=4.0, min=-0.156238, step=0.0398061149425…

In [11]:
qy_min = -3 * sc.Unit('Å^(-1)')
qy_max = 2 * sc.Unit('Å^(-1)')

E_min = -0.05 * sc.Unit('meV')
E_max = 9.0 * sc.Unit('meV')


def update_plot_qyE(amp):
    qx_layer = amp * sc.Unit('Å^(-1)')
    plt.figure(figsize=(6,4))
    p1 = sliced_qxE['qx', qx_layer]['deltaE', E_min:E_max].transpose()#.plot(cmap='jet')
    im = plt.pcolormesh(p1.coords['qy'].values, p1.coords['deltaE'].values, p1.values, cmap='jet', vmin=0, vmax=2)
    cbar = plt.colorbar(im)
    cbar.set_label('I [a.u.]')
    plt.xlim(-1,3)
    plt.ylim(0,7)
    plt.xlabel('$q_y$ [1/Å]')
    plt.ylabel('$\Delta E$ [meV]')
    plt.title(f"$q_x =$ {amp:.2f} Å$^{-1}$")
    #plt.savefig('Pb_phonon_BIFROST_qyE_low.pdf', format='pdf', bbox_inches='tight')
    

w_qyE = ipw.FloatSlider( 
    value=0.17,
    min=min(sliced_qxE.coords['qx'].values),
    max=4,
    step=(sliced_qxE.coords['qx'].values[1]-sliced_qxE.coords['qx'].values[0]),
    description="q_x:",
    disabled=False,
    continuous_update=True,
    orientation="horizontal",
    readout=True,
    readout_format=".2f",
)
# Display the slider


ipw.interactive(update_plot_qyE, amp=w_qyE)

interactive(children=(FloatSlider(value=0.17, description='q_x:', max=4.0, min=-1.171966, step=0.0399662053571…

In [12]:
def to_l(qx):
    c = 3.13
    c_star = 2 * np.pi / c
    return qx / c_star

def to_h(qy):
    a = 4.87
    a_star = 2 * np.pi / a
    return qy / a_star

graph = {'l': to_l}
graph2 = {'h': to_h}

transformed = sliced.transform_coords(['l'], graph=graph)
sliced = transformed.transform_coords(['h'], graph=graph2)

In [13]:
"""
2D Slice at a given DeltaE-value: returns a colorplot with l on the x-axis and h on the y-axis.  
"""
h_min = -2 * sc.Unit('Å^(-1)')
h_max = 4.0 * sc.Unit('Å^(-1)')

E_layer = 5.9 * sc.Unit('meV')

p1_Energy = sliced['deltaE', E_layer]['h', h_min:h_max].transpose()

h_size = sliced.coords['h'].values
print(h_size[1]-h_size[0])

0.03085310559734765


In [14]:
"""
2D Slice at a given h-value: returns a colorplot with l on the x-axis.  
"""
E_min = -0.5 * sc.Unit('meV')
E_max = 8 * sc.Unit('meV')

h_layer = 1 * sc.Unit('Å^(-1)')

p1_1ms = sliced['h', h_layer]['deltaE', E_min:E_max].transpose()

"""
1D Slice at a given l-value: returns a colorplot with l on the x-axis.  
"""

l_slice = -0.25 * sc.Unit('Å^(-1)')

l_size = p1_1ms.coords['l'].values
print(l_size[1]-l_size[0])

p2_1ms = p1_1ms['deltaE', E_min:E_max]['l',l_slice]
E_1ms = p2_1ms.coords['deltaE'].values
E_1ms = (E_1ms[:-1] + E_1ms[1:]) / 2.0



0.01990936390574316


In [ ]:
E_range = np.linspace(0,8,1000)

mpl.rcParams['font.size']        = 15 # standard er 45

fig, ax = plt.subplots(1,3,figsize=(15,3.5))
fig.subplots_adjust(wspace=0.2, hspace=0.3) 

"""
PSC = 1 ms
"""
im0 = ax[0].pcolormesh(p1_Energy.coords['l'].values, p1_Energy.coords['h'].values, p1_Energy.values, cmap='jet', vmin=0, vmax=3) # norm=colors.LogNorm()
#cbar = plt.colorbar(im0)
#cbar.set_label('I [a.u.]')
ax[0].plot(np.linspace(-0.5,1.5,100), np.ones(100), '--', color='white')
ax[0].set_xlim(-0.75,1.75)
ax[0].set_ylim(0,2.5)
ax[0].set_xlabel('$l$ [r.l.u.]')
ax[0].set_ylabel('$h$ [r.l.u.]')
ax[0].set_title(rf"$\Delta E =$ {E_layer.value:.2f} $\pm$ 0.1 [meV]")
ax[0].text(-0.7, 2.25, r'a)', weight='bold', color='black', fontsize=18)



im1 = ax[1].pcolormesh(p1_1ms.coords['l'].values, p1_1ms.coords['deltaE'].values, p1_1ms.values, cmap='jet', vmin=0, vmax=3) # norm=colors.LogNorm()
cbar_1ms = plt.colorbar(im1)
cbar_1ms.ax.set_title('I [a.u.]', fontsize=13, pad=10)
ax[1].plot(np.ones(100)*l_slice.value, np.linspace(3,7,100), '-', color='red',lw=2)
#ax[1].plot(np.ones(100)*l_slice2.value, np.linspace(2.5,5.5,100), '--', color='#B00B55',lw=2)  
ax[1].set_ylim(0,8)
ax[1].set_xlabel('$l$ [r.l.u.]')
ax[1].set_ylabel('$\Delta E$ [meV]')
ax[1].set_title(rf"$h =$ {h_layer.value:.2f} $\pm$ 0.03 [r.l.u]")
ax[1].text(-0.5, 7.2, r'b)', weight='bold', color='black', fontsize=18)



ax[2].errorbar(E_1ms, p2_1ms.values, np.sqrt(p2_1ms.variances),  fmt='o', markerfacecolor='#B00B55', ecolor='#910946', mew=1, markeredgecolor='#910946', label='PSC = 1 ms')
#ax[2].plot(E_range, gauss(E_range, *fit_1ms_1.values), '-', color='#77AC30') 
#ax[2].text(4.05, 0.65, rf'FWHM = {fit_1ms_1.values[2]*2.35:.2f} meV', weight='bold', color="#4E8605", fontsize=11)
ax[2].set_ylabel("I [a.u.]")

#ax[2].errorbar(E_1ms, p2_0p3ms.values, np.sqrt(p2_0p3ms.variances), fmt='^', color='#d95319', ecolor='#b54514', mew=1, markeredgecolor='#b54514', label='PSC = 0.3 ms')
#ax[2].plot(E_range, gauss(E_range, *fit_0p3ms_1.values), linestyle='dotted', color='#d95319', lw=3)
#ax[2].text(5.75, 0.3, rf'FWHM = {fit_0p3ms_1.values[2]*2.35:.2f} meV', weight='bold', color='#d95319', fontsize=11)

ax[2].set_ylabel("I [a.u.]")
ax[2].set_xlim(3,7)
ax[2].set_ylim(-0.03,1.3)
ax[2].grid(linestyle='dotted')
ax[2].set_xlabel(f'$\Delta$ E [meV]')
#ax[2].legend(fontsize=9, loc='upper right')
ax[2].text(3.1, 1.15, r'c)', weight='bold', color='black', fontsize=18)
ax[2].set_title(f'$l =$ -0.25 $\pm$ 0.02 [r.l.u]')

#plt.savefig('../../Figures/PNG/Magnon_zeeman_1T.png', format='png',dpi=200, bbox_inches='tight')
#plt.savefig('../../Figures/SVG/Magnon_zeeman_1T.svg', format='svg',dpi=200, bbox_inches='tight')